# No feasible solution found.

In [ ]:
# import sys
# sys.path
#  /Users/ryantalbot/opt/anaconda3/envs/tf2/bin/python3 -m pip install pyomo

In [1]:
import pandas as pd
import numpy as np
import pulp


In [2]:
# df = pd.read_csv('top_men.csv')

df = pd.read_csv('all_usa.csv')


In [4]:
grouped = df.groupby(['Name', 'Apparatus'])['Score'].mean().reset_index()

In [5]:
df=grouped.copy()

In [6]:
df

,Name,Apparatus,Score
0,alex diab,FX,13.225000
1,alex diab,SR,14.158333
2,alex diab,VT,13.175000
3,alex karadzhov,FX,12.250000
4,alex karadzhov,HB,12.700000
...,...,...,...
441,zachary nunez,HB,13.150000
442,zachary nunez,PB,13.000000
443,zachary nunez,PH,13.133333
444,zachary nunez,SR,12.950000


In [9]:

# Sort the data by 'Score' in descending order
df = df.sort_values(by='Score', ascending=False)

# Initialize variables
team = []
apparatus_count = {}

# Define constraints
max_events_per_person = 6
max_events_per_apparatus = 3
team_size = 5

# Greedy heuristic
for index, row in df.iterrows():
    name = row['Name']
    apparatus = row['Apparatus']

    if name not in team and apparatus_count.get(apparatus, 0) < max_events_per_apparatus:
        if name not in apparatus_count:
            apparatus_count[name] = 1
        else:
            apparatus_count[name] += 1

        team.append(name)

    if len(team) >= team_size:
        break

# Resulting team
selected_team = df[df['Name'].isin(team)]
print(selected_team)


                   Name Apparatus      Score
106     curran phillips        PB  15.156250
95          colt walker        PB  15.019333
276          khoi young        VT  14.860000
391  stephen nedoroscik        PH  14.726143
29           asher hong        PB  14.705556
107     curran phillips        VT  14.520000
98          colt walker        VT  14.441667
32           asher hong        VT  14.424154
31           asher hong        SR  14.199900
273          khoi young        PB  14.100000
271          khoi young        FX  14.000000
97          colt walker        SR  13.911000
27           asher hong        FX  13.825778
274          khoi young        PH  13.460000
105     curran phillips        HB  13.314500
94          colt walker        HB  13.275000
30           asher hong        PH  13.248300
272          khoi young        HB  13.237500
93          colt walker        FX  13.122167
275          khoi young        SR  13.000000
96          colt walker        PH  12.920000
28        

In [10]:
# code started runnin at 11:00pm 11/3/2023

In [16]:
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

# Define the minimum and maximum score values based on your dataset
min_score = 8.683333333333334
max_score = 15.15625

# Assuming your data is in a DataFrame called 'df'
reader = Reader(rating_scale=(min_score, max_score))
data = Dataset.load_from_df(df[['Name', 'Apparatus', 'Score']], reader)
trainset, testset = train_test_split(data, test_size=0.2)

# Train the SVD model
model = SVD()
model.fit(trainset)


In [17]:
from collections import defaultdict

apparatuses = df['Apparatus'].unique()

team_recommendations = defaultdict(list)

for apparatus in apparatuses:
    apparatus_data = df[df['Apparatus'] == apparatus]
    individuals = apparatus_data['Name'].unique()
    
    for individual in individuals:
        predicted_score = model.predict(individual, apparatus).est
        team_recommendations[apparatus].append((individual, predicted_score))

# Sort and select the top 5 individuals for each apparatus
final_team = {}
for apparatus, recommendations in team_recommendations.items():
    recommendations.sort(key=lambda x: x[1], reverse=True)
    final_team[apparatus] = [recommendations[i][0] for i in range(5)]

# Print the final team
for apparatus, team in final_team.items():
    print(f"Apparatus: {apparatus}, Team: {', '.join(team)}")


Apparatus: PB, Team: asher hong, curran phillips, brody malone, colt walker, frederick richard
Apparatus: VT, Team: khoi young, frederick richard, vitaliy guimaraes, colt walker, brody malone
Apparatus: PH, Team: brody malone, vitaliy guimaraes, frederick richard, frederick nathaniel richard, khoi young
Apparatus: FX, Team: frederick richard, brody malone, khoi young, ian lasic-ellis, vitaliy guimaraes
Apparatus: SR, Team: frederick richard, colt walker, brody malone, vitaliy guimaraes, donnell whittenburg
Apparatus: HB, Team: brody malone, vitaliy guimaraes, frederick nathaniel richard, frederick richard, j.r. chou


In [21]:
import pandas as pd



# Create a set of unique apparatus
apparatus_set = set(df['Apparatus'].unique())

# Initialize an empty team and total score
selected_team = []
total_score = 0

# Initialize a dictionary to keep track of how many times each apparatus is used
apparatus_count = {apparatus: 0 for apparatus in apparatus_set}

# Iterate over each apparatus
while len(selected_team) < 6:
    # Sort the dataframe by score in descending order
    sorted_data = df.sort_values(by='Score', ascending=False)
    
    for index, row in sorted_data.iterrows():
        current_apparatus = row['Apparatus']
        
        # Check if the apparatus has not been used 3 times already
        if apparatus_count[current_apparatus] < 3:
            selected_individual = row
            
            # Add the selected individual to the team and update the total score
            selected_team.append(selected_individual)
            total_score += selected_individual['Score']
            
            # Update the apparatus count
            apparatus_count[current_apparatus] += 1
            
            # Remove the selected individual from the dataframe
            df = df.drop(index)
            break

# Print the selected team and the total score
print("Selected Team:")
for i, individual in enumerate(selected_team):
    print(f"Member {i + 1}: {individual['Name']} on {individual['Apparatus']} with score {individual['Score']}")
print(f"Total Score: {total_score}")


Selected Team:
Member 1: colt walker on PB with score 15.019333333333334
Member 2: asher hong on PB with score 14.705555555555556
Member 3: landen paul blixt on VT with score 14.7
Member 4: yul moldauer on PB with score 14.678571428571429
Member 5: dallas hale on VT with score 14.6625
Member 6: kazuki hayashi on VT with score 14.6
Total Score: 88.3659603174603


In [30]:
from ortools.sat.python import cp_model


In [53]:
from ortools.sat.python import cp_model
import pandas as pd


In [ ]:
import pandas as pd
import numpy as np

# Load your dataset
data = df.copy()
# Define your constraints:
# 1. Each person can compete on the same apparatus at most once.
# 2. Each apparatus should be competed on a total of 3 times.
# (You'll need to implement constraint handling accordingly)

# Example of a function to calculate the total score of a team
def calculate_team_score(team):
    return data[data['Name'].isin(team)]['Score'].sum()

# Use optimization algorithms or machine learning models to find the best team
# Implement constraint handling and optimization with the chosen method

# Evaluate and validate the recommended team


In [ ]:
from deap import base, creator, tools, algorithms


In [62]:
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms

# Load your dataset
data = df.copy()

# Define your constraints:
# 1. Each person can compete on the same apparatus at most once.
# 2. Each apparatus should be competed on a total of 3 times.
# (You'll need to implement constraint handling accordingly)

# Create a Fitness class for maximizing the total team score
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Example of a function to calculate the total score of a team
def calculate_team_score(team):
    return data[data['Name'].isin(team)]['Score'].sum(),

# Define the optimization problem
toolbox = base.Toolbox()
toolbox.register("attr", lambda: np.random.choice(data['Name'].unique(), size=5, replace=False))
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.attr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Evaluation function
toolbox.register("evaluate", calculate_team_score)

# Constraints (You'll need to implement constraint handling here)

# Genetic Algorithm parameters
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.1)
toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    population = toolbox.population(n=100)
    CXPB, MUTPB, NGEN = 0.7, 0.2, 50  # Adjust these parameters as needed

    # Evaluate the entire population
    fitnesses = map(toolbox.evaluate, population)
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit

    for gen in range(NGEN):
        # Select the next generation individuals
        offspring = toolbox.select(population, len(population))

        # Clone the selected individuals
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover and mutation
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if np.random.random() < CXPB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        for mutant in offspring:
            if np.random.random() < MUTPB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Replace the old population by the offspring
        population[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in population]
        length = len(population)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5
        print(f"Generation {gen}:")
        print(f"  Min: {min(fits)}")
        print(f"  Max: {max(fits)}")
        print(f"  Avg: {mean}")
        print(f"  Std: {std}")

    best_ind = tools.selBest(population, 1)[0]
    print("Best Team:", best_ind)
    print("Best Team Score:", best_ind.fitness.values[0])
    
    

if __name__ == "__main__":
    main()


Generation 0:
  Min: 268.53249999999997
  Max: 395.9672797619047
  Avg: 353.5733727661781
  Std: 27.233323722271557
Generation 1:
  Min: 285.931
  Max: 401.4661666666667
  Avg: 371.9366242414528
  Std: 20.998631867614012
Generation 2:
  Min: 231.69999999999996
  Max: 402.90616666666665
  Avg: 377.8778426318682
  Std: 27.270607087620352
Generation 3:
  Min: 306.82916666666665
  Max: 407.33449999999993
  Avg: 385.96815598565337
  Std: 20.820149556990337
Generation 4:
  Min: 311.9375
  Max: 411.1645
  Avg: 390.7454057738095
  Std: 21.096754527650248
Generation 5:
  Min: 314.8631130952381
  Max: 411.522
  Avg: 389.2966047619048
  Std: 28.362732358310208
Generation 6:
  Min: 243.93166666666667
  Max: 411.522
  Avg: 386.67252964285717
  Std: 36.727205759189545
Generation 7:
  Min: 243.53616666666665
  Max: 411.522
  Avg: 386.6611984523811
  Std: 36.39951978141286
Generation 8:
  Min: 321.79616666666664
  Max: 411.75261309523813
  Avg: 397.0655439880955
  Std: 26.950696147196375
Generation 9:

In [70]:
import pulp

# Create a PuLP problem
prob = pulp.LpProblem("GymnasticsAssignment", pulp.LpMaximize)

# Define the athletes and apparatus
athletes = df['Name'].unique()
known_apparatus = ['SR', 'PH', 'PB', 'HB', 'FX', 'VT']

# Create binary variables for each athlete and apparatus combination
athlete_vars = pulp.LpVariable.dicts("Athlete", (athletes, known_apparatus), 0, 1, pulp.LpBinary)

# Objective function: Maximize the total average score
athlete_avg_score = {athlete: df[df['Name'] == athlete]['Score'].values[0] for athlete in athletes}
prob += pulp.lpSum(athlete_avg_score[athlete] * pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) for athlete in athletes)

# Constraint 1: Each athlete competes on up to 3 apparatus
for athlete in athletes:
    prob += pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) <= 6  # Allow each athlete to compete on up to 3 apparatus

# Constraint 2: Each apparatus is used exactly three times
for apparatus in known_apparatus:
    prob += pulp.lpSum(athlete_vars[athlete][apparatus] for athlete in athletes) == 5

# Constraint 3: Select exactly 5 athletes
prob += pulp.lpSum(pulp.lpSum(athlete_vars[athlete][apparatus] for apparatus in known_apparatus) for athlete in athletes) == 5

# Solve the problem
prob.solve()

# Display the results
selected_athletes = [athlete for athlete in athletes if any(pulp.value(athlete_vars[athlete][apparatus]) == 1 for apparatus in known_apparatus)]

for apparatus in known_apparatus:
    print(f"Apparatus {apparatus}:")
    for athlete in selected_athletes:
        if any(pulp.value(athlete_vars[athlete][apparatus]) == 1 for apparatus in known_apparatus):
            scores = df[(df['Name'] == athlete) & (df['Apparatus'] == apparatus)]
            if not scores.empty:
                score = scores['Score'].values[0]
                print(f"{athlete} - {score}")
    print()

# Calculate the total average score
total_average_score = sum(athlete_avg_score[athlete] for athlete in selected_athletes)
print(f"Total Average Score: {total_average_score:.6f}")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/366cdab6529b44098eb0755fd534fdaf-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/366cdab6529b44098eb0755fd534fdaf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 93 COLUMNS
At line 3010 RHS
At line 3099 BOUNDS
At line 3586 ENDATA
Problem MODEL has 88 rows, 486 columns and 1458 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

Apparatus SR:
brody malone - 13.9132
shane wiskus - 13.991666666666667
taylor burkhart - 13.459999999999999
taylor troy christo

In [ ]:
dfs=df.copy()

In [ ]:
# Apparatus SR:
# frederick richard - 14.0
# paul juda - 13.95
# yul moldauer - 13.985714

# Apparatus PH:
# brandon dang - 14.1
# stephen nedoroscik - 14.726142857142857
# vitaliy guimaraes - 14.35

# Apparatus PB:
# frederick richard - 14.25825
# vitaliy guimaraes - 13.9
# yul moldauer 14.678571

# Apparatus HB:
# frederick richard - 14.183333333333332
# paul juda - 13.649999999999999
# vitaliy guimaraes - 13.4

# Apparatus FX:
# frederick richard - 14.066666666666668
# paul juda - 14.65
# vitaliy guimaraes - 14.25

# Apparatus VT:
# paul juda - 14.325
# vitaliy guimaraes - 14.45
# yul moldauer - 14.250000

# fr=5
# pj=5
# vg=6
# bd=1
# sn=1



In [63]:
from pulp import LpMaximize, LpProblem, LpVariable

In [67]:
import pandas as pd
from pulp import LpMaximize, LpProblem, LpVariable

data=df.copy()


In [68]:
# Create a list of unique athlete names and assign unique IDs
unique_athletes = data["Name"].unique()
athlete_to_id = {athlete: i + 1 for i, athlete in enumerate(unique_athletes)}

# Create a list of unique apparatus
unique_apparatus = data["Apparatus"].unique()

# Create a matrix to store the performance data
num_athletes = len(unique_athletes)
num_apparatus = len(unique_apparatus)
performance_data = [[0] * num_apparatus for _ in range(num_athletes)]

# Populate the performance data matrix
for _, row in data.iterrows():
    athlete = row["Name"]
    apparatus = row["Apparatus"]
    athlete_id = athlete_to_id[athlete]
    apparatus_index = list(unique_apparatus).index(apparatus)
    if not pd.isna(row["Score"]):
        performance_data[athlete_id - 1][apparatus_index] = row["Score"]

# Define the ILP model
model = LpProblem(name="GymnasticsTeamSelection", sense=LpMaximize)

# Decision variables
x = {(i, j): LpVariable(name=f"x_{i}_{j}", cat="Binary") for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)}

# Objective function
model += sum(performance_data[i - 1][j - 1] * x[i, j] for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)), "Total_Value"

# Constraints
# Constraint 1: Select exactly 5 athletes
model += sum(x[i, j] for i in range(1, num_athletes + 1) for j in range(1, num_apparatus + 1)) == 5

# Constraint 2: Each apparatus is used exactly 3 times
for j in range(1, num_apparatus + 1):
    model += sum(x[i, j] for i in range(1, num_athletes + 1)) == 3

# Constraint 3: Athletes cannot compete on the same apparatus more than once
for i in range(1, num_athletes + 1):
    model += sum(x[i, j] for j in range(1, num_apparatus + 1)) <= 1

# Solve the ILP
model.solve()

# Check if a feasible solution exists
if model.status != 1:
    print("No feasible solution found.")
else:
    # Track the selected team
    selected_team = []

    for i in range(1, num_athletes + 1):
        for j in range(1, num_apparatus + 1):
            if x[i, j].varValue == 1:
                selected_team.append((unique_athletes[i - 1], unique_apparatus[j - 1]))

    # Print the selected team
    print("Selected Gymnastics Team:")
    for athlete, apparatus in selected_team:
        print(f"Athlete: {athlete}, Apparatus: {apparatus}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ryantalbot/opt/anaconda3/envs/tf2/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/52cabbf1b98941ca87feab65ea36fe8a-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/j_/555m2zps099832fjh_m8jjnc0000gn/T/52cabbf1b98941ca87feab65ea36fe8a-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 93 COLUMNS
At line 2936 RHS
At line 3025 BOUNDS
At line 3512 ENDATA
Problem MODEL has 88 rows, 486 columns and 1458 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Problem is infeasible - 0.00 seconds
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.01

No feasible solution found.
